In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [64]:
class LinearRegression:
    def __init__(self, input_dim, output_dim, name = 'LinearRegression'):
        self.name = name
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.weights = np.random.rand(input_dim, self.output_dim)
        self.bias = np.random.rand(1, self.output_dim)

    def __call__(self, x):
        p = self._predict(x)
        return (p >= 0.5).astype(int)

    def __str__(self):
        return f'{self.name}({self.input_dim}, {self.output_dim})'
    
    def _predict(self, x):
        return x @ self.weights + self.bias
    
    def train(self, x, y, epoch = 1000, learning_rate = 1e-1):
        num_samples = x.shape[0]
        last_loss = np.inf

        for i in range(epoch):
            # Step 1: Predict
            p = self._predict(x)

            # Step 2: Calculate errors
            e = p - y

            # Calculate gradients.
            c = 2 / num_samples
            grad_w = c * (x.T @ e)
            grad_b = c * np.sum(e, axis = 0, keepdims = True)

            # Update params.
            self.weights -= learning_rate * grad_w
            self.bias -= learning_rate * grad_b
            
            loss = np.sum(e ** 2) / num_samples

            # If it does not improve, halt the training process.
            if (last_loss - loss) == 0:
                print(f'({i}/{epoch}) Function is optimized, loss was not improved. Done.')
                return
            else:
                last_loss = loss
        
            # Always print the first and the last iteration.
            if i % 10 == 0 or i == epoch - 1:
                print(f'({i}/{epoch}) Loss {loss:.8f}')

In [155]:
dataset = pd.read_csv('./train.csv')
train, test = train_test_split(dataset, test_size = 0.2)

In [ ]:
def clean_cabin(value):
    if value is np.nan:
        return 0
    if len(value.split(' ')) != 1:
        return -1
    letter = value[0]
    number = int(value[1:] or '0')
    return (ord(letter) - 64) * 1000 + number


def clean_data(dataset):
    dataset = dataset.copy()
    drop_columns =  ['Name', 'Ticket', 'Pclass', 'PassengerId', 'Cabin']
    feature_columns = ['Age', 'Fare', 'Sex', 'Parch', 'SibSp', 'E_C', 'E_Q', 'E_S']

    dataset.Age = dataset.Age.fillna(-1) / 1000
    dataset.Fare = dataset.Fare / 1000
    dataset.Sex = dataset.Sex.map(lambda sex: {'male': 1, 'female': 0}[sex])
    dataset.Cabin = dataset.Cabin.map(clean_cabin)
    
    # Embarked
    dataset.Embarked = dataset.Embarked.fillna(dataset['Embarked'].mode()[0])
    dataset = pd.get_dummies(dataset, columns=['Embarked'], prefix='E', dtype=int)

    return dataset[feature_columns].to_numpy(), dataset[['Survived']].to_numpy()

x_train, y_train = clean_data(train)
x_test, y_test = clean_data(test)

In [157]:
model = LinearRegression(input_dim = x_train.shape[1], output_dim = 1)
model.train(x_train, y_train)

error = (model(x_test) - y_test) ** 2
error = error.sum() / len(error)
error = error * 100
print(f'{error:0f}')

(0/1000) Loss 2.73301908
(10/1000) Loss 0.20170392
(20/1000) Loss 0.17488365
(30/1000) Loss 0.16520090
(40/1000) Loss 0.16088853
(50/1000) Loss 0.15875398
(60/1000) Loss 0.15762271
(70/1000) Loss 0.15699527
(80/1000) Loss 0.15663613
(90/1000) Loss 0.15642554
(100/1000) Loss 0.15629933
(110/1000) Loss 0.15622188
(120/1000) Loss 0.15617293
(130/1000) Loss 0.15614082
(140/1000) Loss 0.15611872
(150/1000) Loss 0.15610261
(160/1000) Loss 0.15609013
(170/1000) Loss 0.15607986
(180/1000) Loss 0.15607094
(190/1000) Loss 0.15606287
(200/1000) Loss 0.15605534
(210/1000) Loss 0.15604815
(220/1000) Loss 0.15604121
(230/1000) Loss 0.15603442
(240/1000) Loss 0.15602776
(250/1000) Loss 0.15602120
(260/1000) Loss 0.15601472
(270/1000) Loss 0.15600830
(280/1000) Loss 0.15600195
(290/1000) Loss 0.15599565
(300/1000) Loss 0.15598941
(310/1000) Loss 0.15598323
(320/1000) Loss 0.15597710
(330/1000) Loss 0.15597101
(340/1000) Loss 0.15596498
(350/1000) Loss 0.15595900
(360/1000) Loss 0.15595307
(370/1000) L

/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:18: RuntimeWarning: divide by zero encountered in matmul
  return x @ self.weights + self.bias
/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:18: RuntimeWarning: overflow encountered in matmul
  return x @ self.weights + self.bias
/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:18: RuntimeWarning: invalid value encountered in matmul
  return x @ self.weights + self.bias
/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:33: RuntimeWarning: divide by zero encountered in matmul
  grad_w = c * (x.T @ e)
/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:33: RuntimeWarning: overflow encountered in matmul
  grad_w = c * (x.T @ e)
/var/folders/9x/jyl6_jcn10g17ynbhk1t47wm0000gn/T/ipykernel_35624/905470519.py:33: RuntimeWarning: invalid value encountered in matmul
  grad_w = c * (x.T @ e)
